In [1]:
#%matplotlib inline
#%matplotlib widget
import torch
import gym
import torch
import gym
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from torch.distributions import Normal
from env.WarthogEnvSup import WarthogEnv
import scipy.signal
import time
import pandas as pd
eps = np.finfo(np.float32).eps.item()
#torch.manual_seed(100)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

d:\anaconda\envs\rl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'pandas'

In [ ]:
def discount_cumsum(x, discount):
    """
    magic from rllab for computing discounted cumulative sums of vectors.
    input: 
        vector x, 
        [x0, 
         x1, 
         x2]
    output:
        [x0 + discount * x1 + discount^2 * x2,  
         x1 + discount * x2,
         x2]
    """
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

In [ ]:
class ValueNetwork(nn.Module):
    def __init__(self, obs_dimension, sizes, act = nn.ReLU):
        super(ValueNetwork, self).__init__()
        sizes = [obs_dimension] + sizes + [1]
        out_activation = nn.Identity
        self.layers = []
        for j in range(0,len(sizes) - 1):
            act_l = act if j < len(sizes) -2 else out_activation
            self.layers+=[nn.Linear(sizes[j], sizes[j+1]), act_l()]
        self.v = nn.Sequential(*self.layers)
    def forward(self, x):
        return self.v(x)

In [ ]:
class PolicyNetworkCat(nn.Module):
    def __init__(self, obs_dimension, sizes, action_dimension, act= nn.ReLU):
        super(PolicyNetworkCat, self).__init__()
        sizes = [obs_dimension] + sizes + [action_dimension]
        out_activation = nn.Identity
        self.layers = []
        for j in range(0,len(sizes) - 1):
            act_l = act if j < len(sizes) -2 else out_activation
            self.layers+=[nn.Linear(sizes[j], sizes[j+1]), act_l()]
        self.pi = nn.Sequential(*self.layers)
    def forward(self, x):
        score = self.pi(x)
        #probs = F.softmax(score,dim = 1)
        dist = torch.distributions.Categorical(logits=score)
        return dist

In [ ]:
class PolicyNetworkGauss(nn.Module):
    def __init__(self, obs_dimension, sizes, action_dimension,act = nn.ReLU):
        super(PolicyNetworkGauss, self).__init__()
        sizes = [obs_dimension] + sizes + [action_dimension]
        out_activation = nn.Identity
        self.layers = []
        for j in range(0,len(sizes) - 1):
            act_l = act if j < len(sizes) -2 else out_activation
            self.layers+=[nn.Linear(sizes[j], sizes[j+1]), act_l()]
        self.mu = nn.Sequential(*self.layers)
        log_std = -0.5*np.ones(action_dimension, dtype=np.float32)
        self.log_std = torch.nn.Parameter(torch.as_tensor(log_std), requires_grad=True)
    def forward(self, x):
        mean = self.mu(x)
        std = torch.exp(self.log_std)
        dist = Normal(mean, std)
        return dist

In [ ]:
#env = gym.make('CartPole-v1')
#env = gym.make('CartPole-v1')
env = WarthogEnv('sim_remote_waypoint.txt', 'entropy_manual.csv')
sizes = [128]
obs_dimension = env.observation_space.shape
action_dimension = env.action_space.shape
pi = PolicyNetworkGauss(*obs_dimension, sizes, *action_dimension)
o = torch.zeros(1,42)
a = torch.as_tensor([0.5, 0.3], dtype=torch.float32)
print(o)
m = pi(o)
logp = m.log_prob(a)
print(len(a.shape))
print(m)
print(logp)
print(m.loc)

In [ ]:
class PPOBuffer:
    """
    A buffer for storing trajectories experienced by a PPO agent interacting
    with the environment, and using Generalized Advantage Estimation (GAE-Lambda)
    for calculating the advantages of state-action pairs.
    """

    def __init__(self, obs_dim, act_dim, size, gamma=0.99, lam=0.95):
        self.obs_buf = np.zeros((size, obs_dim), dtype=np.float32)
        #self.act_buf = np.zeros((size,), dtype=np.float32)
        self.act_buf = np.zeros((size,act_dim), dtype=np.float32)
        self.adv_buf = np.zeros(size, dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.ret_buf = np.zeros(size, dtype=np.float32)
        self.val_buf = np.zeros(size, dtype=np.float32)
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, obs, act, rew, val, logp):
        """
        Append one timestep of agent-environment interaction to the buffer.
        """
        assert self.ptr < self.max_size     # buffer has to have room so you can store
        self.obs_buf[self.ptr] = obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.val_buf[self.ptr] = val
        self.logp_buf[self.ptr] = logp
        self.ptr += 1

    def finish_path(self, last_val=0):
        """
        Call this at the end of a trajectory, or when one gets cut off
        by an epoch ending. This looks back in the buffer to where the
        trajectory started, and uses rewards and value estimates from
        the whole trajectory to compute advantage estimates with GAE-Lambda,
        as well as compute the rewards-to-go for each state, to use as
        the targets for the value function.

        The "last_val" argument should be 0 if the trajectory ended
        because the agent reached a terminal state (died), and otherwise
        should be V(s_T), the value function estimated for the last state.
        This allows us to bootstrap the reward-to-go calculation to account
        for timesteps beyond the arbitrary episode horizon (or epoch cutoff).
        """

        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)
       # print(rews)
       # print(vals)
        # the next two lines implement GAE-Lambda advantage calculation
        deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
        self.adv_buf[path_slice] = discount_cumsum(deltas, self.gamma * self.lam)
        
        # the next line computes rewards-to-go, to be targets for the value function
        self.ret_buf[path_slice] = discount_cumsum(rews, self.gamma)[:-1]
        
        self.path_start_idx = self.ptr

    def get(self):
        """
        Call this at the end of an epoch to get all of the data from
        the buffer, with advantages appropriately normalized (shifted to have
        mean zero and std one). Also, resets some pointers in the buffer.
        """
        assert self.ptr == self.max_size    # buffer has to be full before you can get
        self.ptr, self.path_start_idx = 0, 0
        # the next two lines implement the advantage normalization trick

        self.adv_buf = (self.adv_buf - self.adv_buf.mean()) / (self.adv_buf.std()+eps)
        data = dict(obs=self.obs_buf, act=self.act_buf, ret=self.ret_buf,
                    adv=self.adv_buf, logp=self.logp_buf)
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in data.items()}


In [ ]:
def ppo(env, seed = 0, buff_size = 128, batch_size = 4, train_time_steps = 10000000, gamma = 0.99, clip_ratio = 0.2, lr_pi = 3e-5, 
        lr_vf = 1e-3, pi_train_itrs = 4, v_train_itrs = 4, lam = 0.97, max_ep_len = 500, ent_coeff = 0.01):
        obs_dim = env.observation_space.shape
        action_dim = env.action_space.shape
        h_sizes = [64,64]
        vi = ValueNetwork(*obs_dim, h_sizes, act=nn.Tanh).to(device)
        pi = PolicyNetworkGauss(*obs_dim, h_sizes, *action_dim,act=nn.Tanh).to(device)
        data_buff = PPOBuffer(*obs_dim, *action_dim, buff_size)
        policy_opt = optim.Adam(pi.parameters(), lr = lr_pi, eps=1e-5)
        value_opt = optim.Adam(vi.parameters(), lr = lr_vf, eps=1e-5)
        obs = env.reset()
        curr_time_step = 0
        num_episode = 0
        ep_rewards = [0]
        ep_steps = [0]
        start_time = time.time()
        while curr_time_step < train_time_steps: 
                for t in range(0, buff_size):
                        curr_time_step+=1
                        with torch.no_grad():
                                m = pi(torch.as_tensor(obs, dtype=torch.float32).to(device))
                                action = m.sample()
                                action = action.reshape((-1,) + action_dim)
                                logp = m.log_prob(action).sum(dim=1)
                                action = action.cpu().numpy() 
                                clipped_action = np.clip(action, env.action_space.low, env.action_space.high)
                                obs_new, rew, done, _ = env.step(clipped_action[0])
                                ep_rewards[num_episode]+=rew 
                                ep_steps[num_episode]+=1
                                v = vi(torch.as_tensor(obs, dtype=torch.float32).to(device))
                        data_buff.store(obs, action, rew, v.cpu().numpy(), logp.cpu().numpy())
                        obs = obs_new
                        if done or buff_size-1:
                                if done:
                                        v_ = 0.
                                        obs = env.reset()
                                        done = False
                                        num_episode+=1
                                        ep_rewards.append(0)
                                        ep_steps.append(0)
                                        curr_time = time.time()
                                        if num_episode %100 == 0:
                                                print(f'episode: {num_episode-1} \t episode_reward: {np.mean(ep_rewards[-10:-2])} \t total steps:{curr_time_step}\t fps"{curr_time_step/(curr_time-start_time)}\t avg_ep_steps: {np.mean(ep_steps[-10:-2])}')
                                                torch.save(pi, f'./temp_policy/tan_h/manaul_entropy3_ppo_batch_{curr_time_step}_rew_{int(np.mean(ep_rewards[-10:-2]))}.pt')
                                else:
                                        v_ = vi(torch.as_tensor(obs, dtype=torch.float32).to(device))
                                        v_ = v_.detach().cpu().numpy()
                                data_buff.finish_path(v_)
                        if(curr_time_step % 100000 == 0):
                                np.savetxt(f'./temp_policy/tan_h/avg_rew_{curr_time_step}', ep_rewards, fmt='%f')

                data = data_buff.get()
                ret, act, adv, o, logp_old= data['ret'].to(device), data['act'].to(device), data['adv'].to(device), data['obs'].to(device), data['logp'].to(device)
                for j in range(0, pi_train_itrs):
                        indices = np.random.permutation(buff_size)
                        start_idx = 0
                        while start_idx < buff_size:

                                batch_ind = indices[start_idx: start_idx+batch_size]
                                ret_b = ret[batch_ind]
                                act_b = act[batch_ind]
                                adv_b = adv[batch_ind]
                                o_b = o[batch_ind]
                                logp_old_b = logp_old[batch_ind]
                                act_dist = pi(o_b)
                                logp = act_dist.log_prob(act_b).sum(dim=1)
                                ratio = torch.exp(logp - logp_old_b)
                                clip_adv = torch.clamp(ratio, 1-clip_ratio, 1+clip_ratio) * adv_b
                                loss_pi = -(torch.min(ratio * adv_b, clip_adv)).mean() - ent_coeff* torch.mean(-logp)
                                policy_opt.zero_grad()
                                loss_pi.backward()
                                policy_opt.step()
                                val = vi(o_b)
                                value_loss = F.mse_loss(val.flatten(), ret_b)
                                value_opt.zero_grad()
                                value_loss.backward()
                                value_opt.step()
                                start_idx = start_idx+batch_size
        return pi



In [9]:
#python.dataScience.textOutputLimit = 0
pi = ppo(env)


In [ ]:
#env.clear()

#%matplotlib widget
#env = WarthogEnv('sim_remote_waypoint.txt', None)
#pi = torch.load("./temp_warthog.pt")
#plt.ion()
#plt.pause(2)
#obs = env.reset()
#for i in range(0,400):
#    m = pi(torch.as_tensor(obs, dtype=torch.float32).to(device))
#    action = m.sample()
#    obs, rew, done, info = env.step(action.cpu().numpy())
#    env.render()
#    if done:
#        obs = env.reset()


In [ ]:
#torch.save(pi,"temp_warthog.pt")